In [1]:
import numpy as np
import dedalus.public as d3

import h5py
import logging
logger = logging.getLogger(__name__)

import matplotlib.pyplot as plt

In [2]:
π      = np.pi
ri, ro = 0.89, 0.99
volume = 4*π * (ro**3 - ri**3 )/3

nHz = 2*π * 1e-9
day = 3600 * 24
Ω0  = 466  * nHz 

μ0 = 4*π
ρ0 = 0.0309

rsun    = 696e8
u_units = rsun*nHz
b_units = np.sqrt(μ0*ρ0)*u_units

dtype   = np.complex128
φθr     = d3.SphericalCoordinates('φ', 'θ', 'r')
dist    = d3.Distributor(φθr,dtype=dtype)
basis   = d3.ShellBasis(φθr, (2,384,64), radii=(ri,ro), dtype=dtype, dealias=3*(1,))
φ, θ, r = basis.global_grids(3*(1,))

In [3]:
#load data
case = ['te601_P292', 'te60_Pnone'][1]

load_from_file = True

eig = np.load('polished_' + case + '/eig.npy')
γ, ω = eig.imag, eig.real

file = h5py.File('polished_' + case + '/checkpoints/checkpoints_s1.h5')

growth =   1 / (γ*Ω0*day)
period = 2*π / (ω*Ω0*day)

if period > 1000*growth:
    period, gap = 'P = None', ', '
else:
    period, gap = f'P = {period:.0f} day', ',  '

growth = f'τ = {growth:.0f} day'

print(case + ':')
print(f'γ = {γ:.4f}' + gap + f'ω = {ω:.4f}')
print(growth + ', ' + period)

u = dist.VectorField(φθr, bases=basis, name='uu')
b = dist.VectorField(φθr, bases=basis, name='bb')

u.load_from_hdf5(file,0)
b.load_from_hdf5(file,0)

u.name = 'u'
b.name = 'b'

b['g'] *= b_units/np.max(np.abs(u['g']))
u['g'] *= u_units/np.max(np.abs(u['g']))

# auxiliary data

grad    = d3.grad
curl    = d3.curl
div     = d3.div
angular = d3.angular
integ   = d3.integ
lift    = lambda A: d3.Lift(A, basis.clone_with(k=1), -1)

ρ    = dist.Field(bases=basis.meridional_basis, dtype=dtype,name='ρ')
dρ   = dist.VectorField(φθr,bases=basis.meridional_basis, dtype=dtype,name='dρ')
rvec = dist.VectorField(φθr,bases=basis.meridional_basis,name='rvec')

z = (2*r-ri-ro)/(ro-ri)

ρ_coefs    = [ 0.4287056, -0.50122643, 0.07841967, 0.00206746, 0.00243878]
ρ['g']     = ρ0 * np.polynomial.chebyshev.chebval(z,ρ_coefs)

dρ_coefs   = [-9.78493705, 6.46519649, 0.45630896, 0.24012609, 0.11130384] 
dρ['g'][2] = ρ0 * np.polynomial.chebyshev.chebval(z,dρ_coefs)

rvec['g'][2] = r

τi = dist.VectorField(φθr,bases=basis.S2_basis(radius=ri),dtype=dtype,name='τi')
τo = dist.VectorField(φθr,bases=basis.S2_basis(radius=ro),dtype=dtype,name='τo')
τχ = dist.Field(dtype=dtype,name='τχ')
χ  = dist.Field(bases=basis,dtype=dtype,name='χ')

def size(x,y=None):
    s = np.max(np.abs(x.evaluate()['g']))
    if y: s /= np.average(np.abs(y.evaluate()['g']))
    print(f"{s:.1e}")

te60_Pnone:
γ = 0.0656, ω = 0.0000
τ = 60 day, P = None


In [4]:
# curl(a) = b

a  = dist.VectorField(φθr, bases=basis, dtype=dtype, name='a')
ψ  = dist.VectorField(φθr, bases=basis, dtype=dtype, name='ψ')


if load_from_file:
    
    a['g'] = np.load(open(case + '_a_vector.npy',   'rb'))
    ψ['g'] = np.load(open(case + '_psi_vector.npy', 'rb'))
    
else:

    da = grad(a) + rvec*lift(τi)

    problem = d3.LBVP([a,χ,τi,τo,τχ],namespace=locals())
    problem.add_equation("trace(da) + τχ = 0")
    problem.add_equation("div(da) + grad(χ) + lift(τo) = -curl(b)")
    problem.add_equation("angular(a(r=ri)) = 0")
    problem.add_equation("angular(a(r=ro)) = 0")
    problem.add_equation("χ(r=ri)  = 0")
    problem.add_equation("χ(r=ro)  = 0")
    problem.add_equation("integ(χ) = 0")

    solver = problem.build_solver()
    solver.solve()

    # curl(ρψ) = ρu


    dψ = grad(ρ*ψ) + rvec*lift(τi)

    problem = d3.LBVP([ψ,χ,τi,τo,τχ],namespace=locals())
    problem.add_equation("trace(dψ) + τχ = 0")
    problem.add_equation("div(dψ) + grad(χ) + lift(τo) = - curl(ρ*u)")
    problem.add_equation("angular(ψ(r=ri)) = 0")
    problem.add_equation("angular(ψ(r=ro)) = 0")
    problem.add_equation("χ(r=ri)  = 0")
    problem.add_equation("χ(r=ro)  = 0")
    problem.add_equation("integ(χ) = 0")

    solver = problem.build_solver()
    solver.solve()
    
    np.save(open(case + '_a_vector.npy','wb'),   a['g'])
    np.save(open(case + '_psi_vector.npy','wb'), ψ['g'])
    
    size(curl(a) - b, b)
    size(curl(ρ*ψ) - ρ*u, ρ*u)

In [5]:
eφ = dist.VectorField(φθr,bases=basis.meridional_basis,dtype=dtype,name='eφ')
eθ = dist.VectorField(φθr,bases=basis.meridional_basis,dtype=dtype,name='eθ')
er = dist.VectorField(φθr,bases=basis.meridional_basis,dtype=dtype,name='er')

eφ['g'][0] += 1
eθ['g'][1] += 1
er['g'][2] += 1

aφ = (eφ @ a).evaluate()
uφ = (eφ @ u).evaluate()
ψφ = (eφ @ ψ).evaluate()
bφ = (eφ @ b).evaluate()

In [6]:
aa0 = (np.abs(aφ)**2).evaluate()
au0 = (np.conj(aφ)*uφ + np.conj(uφ)*aφ).evaluate()

aa0['c'][0,1:,:] *= 0
au0['c'][0,1:,:] *= 0

In [9]:
r0, r1 = rsun * ri, rsun * ro

def B0(r):
    return (1 + 4*(r/r1)**5)/(5*(r/r1)**3) * 466 * b_units * 10**(-11/4)

Ψ  = (er @ u).evaluate()
Ψ['g'] *= (rsun*r)**(1/2) * B0(r*rsun)

Ψr = er @ grad(Ψ)/rsun
Ψθ = eθ @ grad(Ψ)/rsun

ΨΨ0 = (np.abs(Ψ)**2).evaluate()
ΨΨ0['c'][0,1:,:] *= 0

ΨrΨr0 = (np.abs(Ψr)**2).evaluate()
ΨrΨr0['c'][0,1:,:] *= 0

ReΨrΨc0 = ( 0.5*(Ψr*np.conj(Ψ)+Ψ*np.conj(Ψr)) ).evaluate()
ReΨrΨc0['c'][0,1:,:] *= 0

ΨθΨθ0 = (np.abs(Ψθ)**2).evaluate()
ΨθΨθ0['c'][0,1:,:] *= 0

In [14]:
rcoord = d3.Coordinate('r')
rdist  = d3.Distributor(rcoord, dtype=np.complex128)
rbasis = d3.Chebyshev(rcoord, size=64, bounds=(r0, r1))

dr     = lambda A: d3.Differentiate(A, rcoord)

ΨΨ       = rdist.Field(bases=rbasis, name='ΨΨ')
ΨΨ['g']  = ΨΨ0['g'][0,0]

ΨrΨr       = rdist.Field(bases=rbasis, name='ΨrΨr')
ΨrΨr['g']  = ΨrΨr0['g'][0,0]

ΨθΨθ      = rdist.Field(bases=rbasis, name='ΨθΨθ')
ΨθΨθ['g']  = ΨθΨθ0['g'][0,0]

ReΨrΨc = rdist.Field(bases=rbasis, name='ReΨrΨc')
ReΨrΨc['g'] = ReΨrΨc0['g'][0,0]

#Setup background
def ρ0(r):
    z = (r - r0) / (r1 - r0)
    a = [0.031256,0.053193,0.033703,0.023766,0.012326]
    return sum(a[k]*(-z)**k for k in range(len(a)))

def Ω0(r):
    z = (r - r0) / (r1 - r0)
    return (1 + 0.02*z - 0.01*z**2 - 0.03*z**3) * 466 * nHz

r      = rdist.Field(bases=rbasis, name='r')
r['g'] = rdist.local_grid(rbasis)

ρ      = rdist.Field(bases=rbasis, name='ρ')
ρ['g'] = ρ0(r['g'])

B      = rdist.Field(bases=rbasis, name='B')
B['g'] = B0(r['g'])

Ω      = rdist.Field(bases=rbasis, name='Ω')
Ω['g'] = Ω0(r['g'])

S = - r * dr(Ω)

aa = rdist.Field(bases=rbasis, name='aa')
aa['g'] = aa0['g'][0,0] * rsun**2

au = rdist.Field(bases=rbasis, name='au')
au['g'] = au0['g'][0,0] * rsun

L = (2 * B * au - (2*Ω - S) * aa) / ( 2 * B**2 ) 
Φ = aa/( 2 * B )

δΩ = dr( r**2 * ρ * L ) / ( r**3 * ρ )
δA = dr( r**2 * ρ * Φ ) / ( r**2 * ρ )

δB =       dr( r * δA ) / r
δS = - r * dr(     δΩ )

V1 =  - 2 * Ω * S * μ0 * ρ / B**2 
V2 =  dr( dr(B) / ( r * ρ ) ) * r * ρ / B 
V3 = 3 / ( 4 * r**2 ) 
V  = V1 + V2 + V3

δV11 =       (δΩ/Ω) * V1 
δV12 =       (δS/S) * V1
δV13 =  -2 * (δB/B) * V1

δV21 =     - (δB/B) * V2 
δV22 = dr( dr(δB) / ( r * ρ ) ) * r * ρ / B

δV = δV11 + δV12 + δV13 + δV21 + δV22

In [15]:
integ = lambda f: d3.integ(f).evaluate()['g'][0].real

f0 = ΨrΨr + ΨθΨθ + V * ΨΨ
F0 = integ(f0)

β  = 2*δB/B
dβ = dr(β)

f1 = dβ * ReΨrΨc + β * f0 + δV * ΨΨ
F1 = integ(f1)

amp = (-F0/F1)**0.5
print(f'amplitude: {amp:.1f} nHz')

amplitude: 6.4 nHz
